# Case Study: How Does Bike-Share Navigate Speedy Success

## Pre-Processing

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt 

In [59]:
#read the path
file_path = "/Users/pq/Documents/da_projects/1_da_cyclist/data/"

#list all the files from the directory
all_files = glob.glob(os.path.join(file_path , "*.csv"))

#read all csv files into dataframe
ls = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    ls.append(df)

df = pd.concat(ls, axis=0, ignore_index=True)

print(f"Shape of Dataframe: {df.shape}")
df.head()

Shape of Dataframe: (6087762, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,550CF7EFEAE0C618,electric_bike,2022-08-07 21:34:15,2022-08-07 21:41:46,NaN,NaN,NaN,NaN,41.93,-87.69,41.94,-87.72,casual
1,DAD198F405F9C5F5,electric_bike,2022-08-08 14:39:21,2022-08-08 14:53:23,NaN,NaN,NaN,NaN,41.89,-87.64,41.92,-87.64,casual
2,E6F2BC47B65CB7FD,electric_bike,2022-08-08 15:29:50,2022-08-08 15:40:34,NaN,NaN,NaN,NaN,41.97,-87.69,41.97,-87.66,casual
3,F597830181C2E13C,electric_bike,2022-08-08 02:43:50,2022-08-08 02:58:53,NaN,NaN,NaN,NaN,41.94,-87.65,41.97,-87.69,casual
4,0CE689BB4E313E8D,electric_bike,2022-08-07 20:24:06,2022-08-07 20:29:58,NaN,NaN,NaN,NaN,41.85,-87.65,41.84,-87.66,casual


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6087762 entries, 0 to 6087761
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 603.8+ MB


In [61]:
#convert start_at and ended_at to datetime 
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

#calculate duration in minutes 
df['duration_in_mins'] = (df['ended_at']-df['started_at']).astype('timedelta64[m]')

#sanity check
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_mins
0,550CF7EFEAE0C618,electric_bike,2022-08-07 21:34:15,2022-08-07 21:41:46,NaN,NaN,NaN,NaN,41.93,-87.69,41.94,-87.72,casual,7.0
1,DAD198F405F9C5F5,electric_bike,2022-08-08 14:39:21,2022-08-08 14:53:23,NaN,NaN,NaN,NaN,41.89,-87.64,41.92,-87.64,casual,14.0
2,E6F2BC47B65CB7FD,electric_bike,2022-08-08 15:29:50,2022-08-08 15:40:34,NaN,NaN,NaN,NaN,41.97,-87.69,41.97,-87.66,casual,10.0
3,F597830181C2E13C,electric_bike,2022-08-08 02:43:50,2022-08-08 02:58:53,NaN,NaN,NaN,NaN,41.94,-87.65,41.97,-87.69,casual,15.0
4,0CE689BB4E313E8D,electric_bike,2022-08-07 20:24:06,2022-08-07 20:29:58,NaN,NaN,NaN,NaN,41.85,-87.65,41.84,-87.66,casual,5.0


In [63]:
df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    886328
start_station_id      886460
end_station_name      947476
end_station_id        947617
start_lat                  0
start_lng                  0
end_lat                 6121
end_lng                 6121
member_casual              0
duration_in_mins           0
dtype: int64

In [62]:
df['member_casual'].value_counts()

member    3660441
casual    2427321
Name: member_casual, dtype: int64

In [64]:
#handling missing data - drop all records when any one of the end_lat or end_lng is null as we won't be able to calculate distance
cols_missingvalues = ['end_lat', 'end_lng']
df_cleaned = df.dropna(axis=0, subset=cols_missingvalues)

In [65]:
print(f"Percentage of data remaining after cleaning: {len(df_cleaned)/len(df) * 100:.2f}%")
print(f"Shape of cleaned dataframe: {df_cleaned.shape}")

Percentage of data remaining after cleaning: 99.90%
Shape of cleaned dataframe: (6081641, 14)


In [66]:
df_cleaned['member_casual'].value_counts()

member    3659726
casual    2421915
Name: member_casual, dtype: int64

In [67]:
#calculate distance travelled 
from geopy import distance

def Haversine(lat1,lon1,lat2,lon2, **kwarg):
    """
    This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, 
    the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points 
    (ignoring any hills they fly over, of course!).
    Haversine
    formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    c = 2 ⋅ atan2( √a, √(1−a) )
    d = R ⋅ c
    where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
    note that angles need to be in radians to pass to trig functions!
    """
    R = 6371.0088
    lat1,lon1,lat2,lon2 = map(np.radians, [lat1,lon1,lat2,lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) **2
    c = 2 * np.arctan2(a**0.5, (1-a)**0.5)
    d = R * c
    return round(d,4)

df_cleaned['distance_km'] = df_cleaned.apply(lambda row : Haversine(row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']), axis = 1)
df_cleaned.head()

/var/folders/hl/nzp7lrk160zgns5knsssn_5c0000gn/T/ipykernel_2648/3069554401.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['distance_km'] = df_cleaned.apply(lambda row : Haversine(row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']), axis = 1)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_mins,distance_km
0,550CF7EFEAE0C618,electric_bike,2022-08-07 21:34:15,2022-08-07 21:41:46,NaN,NaN,NaN,NaN,41.93,-87.69,41.94,-87.72,casual,7.0,2.7193
1,DAD198F405F9C5F5,electric_bike,2022-08-08 14:39:21,2022-08-08 14:53:23,NaN,NaN,NaN,NaN,41.89,-87.64,41.92,-87.64,casual,14.0,3.3359
2,E6F2BC47B65CB7FD,electric_bike,2022-08-08 15:29:50,2022-08-08 15:40:34,NaN,NaN,NaN,NaN,41.97,-87.69,41.97,-87.66,casual,10.0,2.4802
3,F597830181C2E13C,electric_bike,2022-08-08 02:43:50,2022-08-08 02:58:53,NaN,NaN,NaN,NaN,41.94,-87.65,41.97,-87.69,casual,15.0,4.6977
4,0CE689BB4E313E8D,electric_bike,2022-08-07 20:24:06,2022-08-07 20:29:58,NaN,NaN,NaN,NaN,41.85,-87.65,41.84,-87.66,casual,5.0,1.3866


In [7]:
# New attribute - day of the week
df_cleaned['day_of_week'] = pd.to_datetime(df_cleaned['started_at']).dt.dayofweek

days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
df_cleaned['day_of_week_named'] = df_cleaned['day_of_week'].apply(lambda x: days[x])

# sanity check 
df_cleaned.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_mins,distance_km,day_of_week,day_of_week_named
0,550CF7EFEAE0C618,electric_bike,2022-08-07 21:34:15,2022-08-07 21:41:46,NaN,NaN,NaN,NaN,41.93,-87.69,41.94,-87.72,casual,7.0,2.7193,6,Sun
1,DAD198F405F9C5F5,electric_bike,2022-08-08 14:39:21,2022-08-08 14:53:23,NaN,NaN,NaN,NaN,41.89,-87.64,41.92,-87.64,casual,14.0,3.3359,0,Mon
2,E6F2BC47B65CB7FD,electric_bike,2022-08-08 15:29:50,2022-08-08 15:40:34,NaN,NaN,NaN,NaN,41.97,-87.69,41.97,-87.66,casual,10.0,2.4802,0,Mon
3,F597830181C2E13C,electric_bike,2022-08-08 02:43:50,2022-08-08 02:58:53,NaN,NaN,NaN,NaN,41.94,-87.65,41.97,-87.69,casual,15.0,4.6977,0,Mon
4,0CE689BB4E313E8D,electric_bike,2022-08-07 20:24:06,2022-08-07 20:29:58,NaN,NaN,NaN,NaN,41.85,-87.65,41.84,-87.66,casual,5.0,1.3866,6,Sun


In [68]:
# sanity check
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6081641 entries, 0 to 6087761
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  duration_in_mins    float64       
 14  distance_km         float64       
dtypes: datetime64[ns](2), float64(6), object(7)
memory usage: 742.4+ MB


In [8]:
#export cleaned data into csv to preload for future analysis
df_cleaned.to_csv('/Users/pq/Documents/da_projects/1_da_cyclist/data/publictripdata_202303_202203.csv', index=False)

## Pandas Profile Report Generation

In [2]:
df_cleaned = pd.read_csv('/Users/pq/Documents/da_projects/1_da_cyclist/data/publictripdata_202303_202203.csv')

In [4]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_cleaned)
profile.to_file("Analysis.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Refer to cyclistic_analysis.ipynb for further analysis and visualisation after data cleaning.

Total duration: 2hr (including environment setup)